# Abstract

In [1]:
#importing libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1                        # Number of parallel threads used to run Algorithms
min_mem_size=6 
classification= True
scale=False
max_models=None    
model_path=None
balance_y=False 
balance_threshold=0.2             
name=None 
server_path=None  
analysis=0 

When enabled, H2O will either undersample the majority classes or oversample the minority classes. Note that the resulting model will also correct the final probabilities (“undo the sampling”) using a monotonic transform, so the predicted probabilities of the first model will differ from a second model. However, because AUC only cares about ordering, it won’t be affected.

In [3]:
data_path='C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv'

### Data Cleaning

In [4]:
import pandas as pd
data= pd.read_csv("Employee_access_data.csv", decimal = ',')

In [5]:
data.dtypes

ACTION              int64
RESOURCE            int64
MGR_ID              int64
ROLE_ROLLUP_1       int64
ROLE_ROLLUP_2       int64
ROLE_DEPTNAME       int64
ROLE_TITLE          int64
ROLE_FAMILY_DESC    int64
ROLE_FAMILY         int64
ROLE_CODE           int64
dtype: object

**Checking for the Null values**

In [6]:
total = data.isnull().sum()[data.isnull().sum() != 0].sort_values(ascending = False)
percent = pd.Series(round(total/len(data)*100,2))
pd.concat([total, percent], axis=1, keys=['total_missing', 'percent'])

,total_missing,percent


No null values

### RUNTIME

300, 500, 700, 1000, 1200

In [7]:
run_time= 1200

In [8]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size, "GB")

2 GB


In [9]:
#Defining functions

#generating random run_id
def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str

# storing in m_data dictionary  
def set_meta_data(analysis,run_id,server,data,model_path,run_time,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  # m_data['target']=target
  #m_data['classification']=classification
  m_data['project'] =name
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['start_time_sec'] = time.time()
  m_data['min_mem_size'] = min_mem_size
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold 
  m_data['max_models']=model
  m_data['scale']=scale  
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  
  m_data['analysis'] = analysis
  m_data['end_time_sec'] = time.time()  
  return m_data

#converting dictionary to json
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
# Segregating in different lists of int, enum, reals and checking for missing values and then scaling(standardizing)    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return

# Determining Independent variables (X) from the dataset 
def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x

#Checkinh if X does not exist then remove
def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    

### RUN_ID

In [10]:
#randomly generating run_id through alphabet function
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
a = run_id + '_EmpAccess_' + str(run_time)
run_dir = os.path.join(server_path,a)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id) 

kfOyShRCX


In [11]:
# # Creating a log file
# logfile=run_id+'_autoh2o_log.zip'
# logs_path=os.path.join(run_dir,'logs')
# print(logs_path,' ',logfile)

In [12]:
name = run_id+'_EmpAccess_' + str(run_time)
name

'kfOyShRCX_EmpAccess_1200'

In [13]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,model_path,run_time,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)  

{'project': 'kfOyShRCX_EmpAccess_1200', 'run_time': 1200, 'run_id': 'kfOyShRCX', 'start_time_sec': 1555603544.8401034, 'min_mem_size': 2, 'balance': False, 'balance_threshold': 0.2, 'max_models': None, 'scale': False, 'model_path': None, 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project', 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv', 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\kfOyShRCX_EmpAccess_1200', 'nthreads': 1, 'analysis': 0, 'end_time_sec': 1555603544.8401034}


In [14]:
if classification :
    meta_data["Problem_type"] = "Classification"
    print("Problem Type:  Classification")
else:
    meta_data["Problem_type"] = "Regression"
    print("Problem Type:  Regression")

Problem Type:  Classification


In [15]:
# 65535 Highest port no
port_no=random.randint(5555,55555)

#initializing H2O
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:23039 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from C:\Users\hp\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\hp\AppData\Local\Temp\tmpdrwff5a4
  JVM stdout: C:\Users\hp\AppData\Local\Temp\tmpdrwff5a4\h2o_hp_started_from_python.out
  JVM stderr: C:\Users\hp\AppData\Local\Temp\tmpdrwff5a4\h2o_hp_started_from_python.err
  Server is running at http://127.0.0.1:23039
Connecting to H2O server at http://127.0.0.1:23039 ... successful.


H2O cluster uptime:,09 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_hp_mgt949
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [16]:
print(data_path)              #path of csv 

C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv


In [17]:
#importing data file on h2o server
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
df.head()

ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
1,36724,14457,118219,118220,117884,117879,267952,19721,117880
1,36135,5396,117961,118343,119993,118321,240983,290919,118322
1,42680,5905,117929,117930,119569,119323,123932,19793,119325
0,45333,14561,117951,117952,118008,118568,118568,19721,118570
1,25993,17227,117961,118343,123476,118980,301534,118295,118982
1,19666,4209,117961,117969,118910,126820,269034,118638,126822
1,31246,783,117961,118413,120584,128230,302830,4673,128231
1,78766,56683,118079,118080,117878,117879,304519,19721,117880


In [19]:
rows = len(df)
print("Total rows in the data set = ", rows)
cols = len(df.columns)
print("Total Columns in the data set = ", cols)

rowscols = df.shape
shape = rows * cols
print("rows X columns = ", rowscols)
print("Total Records = ", shape)

meta_data['Total Records'] = shape

Total rows in the data set =  32769
Total Columns in the data set =  10
rows X columns =  (32769, 10)
Total Records =  327690


In [20]:
df.describe()

Rows:32769
Cols:10




,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
type,int,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,25.0,4292.0,23779.0,4674.0,117879.0,4673.0,3130.0,117880.0
mean,0.9421099209618847,42923.916170771125,25988.957978577324,116952.62778845867,118301.82315603165,118912.77991394309,125916.15264426745,170178.36964814307,183703.40889255097,119789.43013213709
maxs,1.0,312153.0,311696.0,311178.0,286791.0,286792.0,311867.0,311867.0,308574.0,270691.0
sigma,0.23353903780676308,34173.892702138255,35928.03165014073,10875.563591093745,4551.588572012568,18961.32291708769,31036.465824743256,69509.46213013002,100488.40741337684,5784.275515531029
zeros,1897,13,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0
0,1.0,39353.0,85475.0,117961.0,118300.0,123472.0,117905.0,117906.0,290919.0,117908.0
1,1.0,17183.0,1540.0,117961.0,118343.0,123125.0,118536.0,118536.0,308574.0,118539.0
2,1.0,36724.0,14457.0,118219.0,118220.0,117884.0,117879.0,267952.0,19721.0,117880.0


In [21]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

### Depentdent Variable

In [22]:
# dependent variable
# assign target an d inputs for classification or regression
if target==None:
    target="ACTION"   
y = target
meta_data['Target']=y
y

'ACTION'

### Independent Variables

In [23]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [24]:
# independent variables
# putting all independent variables in the list X 

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
  print(X)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables to meta data

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']


### Problem Type

The dependent variable for this data set is of Binary/ classification type.

In [25]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [26]:
df[y]

ACTION
1
1
1
1
1
0
1
1
1
1


**Checking the number of classes of dependent variable**

In [27]:
# Total categories in the target column
lvl = df[y].levels()
print(lvl)
meta_data["levels"] = lvl

[['0', '1']]


y has 2 levels or classes

In [28]:
# checking value of target for real, int, enum
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok   

In [29]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [30]:
# since y is enum type
print(ok)

True


In [31]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    

In [32]:
# getting the data types of all the variables in the dataset
allV=get_variables_types(df)
allV

{'ACTION': 'enum',
 'RESOURCE': 'int',
 'MGR_ID': 'int',
 'ROLE_ROLLUP_1': 'int',
 'ROLE_ROLLUP_2': 'int',
 'ROLE_DEPTNAME': 'int',
 'ROLE_TITLE': 'int',
 'ROLE_FAMILY_DESC': 'int',
 'ROLE_FAMILY': 'int',
 'ROLE_CODE': 'int'}

In [33]:
# Adding the data types to meta data 
meta_data['variables']=allV

### Using H2OAutoML

* Used for automatic training and tuning of the models

In [34]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [35]:
model_start_time = time.time()

In [36]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [37]:
execution_time =  time.time() - model_start_time
meta_data['model_execution_time_sec'] = execution_time
print("Execution time for ", run_time,"sec =  ",meta_data['model_execution_time_sec'])

Execution time for  1200 sec =   2760.7395112514496


## LeaderBoard

In [38]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [39]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_AutoML_20190418_120619,0.856369,0.156949,0.371744,0.200741,0.040297
1,StackedEnsemble_BestOfFamily_AutoML_20190418_1...,0.854217,0.157098,0.326397,0.200571,0.040229
2,GBM_grid_1_AutoML_20190418_120619_model_2,0.853118,0.154296,0.328587,0.200303,0.040121
3,GBM_grid_1_AutoML_20190418_120619_model_5,0.850342,0.160961,0.365139,0.205497,0.042229
4,DRF_1_AutoML_20190418_120619,0.847918,0.178072,0.359026,0.201379,0.040553
5,GBM_grid_1_AutoML_20190418_120619_model_9,0.845875,0.160482,0.359021,0.204446,0.041798
6,GBM_4_AutoML_20190418_120619,0.845188,0.159094,0.362836,0.202633,0.041060
7,XRT_1_AutoML_20190418_120619,0.843907,0.171061,0.338773,0.202070,0.040832
8,GBM_3_AutoML_20190418_120619,0.839115,0.162473,0.393567,0.205262,0.042133
9,GBM_5_AutoML_20190418_120619,0.834543,0.165616,0.419586,0.207439,0.043031


In [40]:
length = len(aml_leaderboard_df)- 1
length
meta_data["Models_generated"] = length

### Saving Leaderboard into csv

In [41]:
# save leaderboard to csv
# run_time = run_time.ascharacter()
leaderboard_stats=run_id+'EmpAccess_'+ str(run_time) + '_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

### Generating hyperparameters for the best model on the leaderboard

In [42]:
# STart best model as first model
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [43]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_AllModels_AutoML_20190418_120619',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_AllModels_AutoML_20190418_120619'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_py_6_sid_8fd2',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_py_6_sid_8fd2'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ColSpecifierV3',
    'schema_type': 'VecSpecifier'},
   'column_name': 'ACTION',
   'is_member_of_frames': None}},
 'validation_frame': {'default': None, 'actual': None},
 'blending_frame': {'default': None, 'actual': None},
 'base_models': {'default': [],
  'actual': [{'__meta': {'schema_version': 3,
     'schema_nam

In [44]:
# generating normalized coeff.
mods=mod_best.coef_norm
print(mods)


Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190418_120619
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.01616292634887272
RMSE: 0.12713349813826694
LogLoss: 0.07258968128495598
Null degrees of freedom: 32768
Residual degrees of freedom: 32758
Null deviance: 14491.899767390338
Residual deviance: 4757.382532053446
AIC: 4779.382532053446
AUC: 0.9949236635825064
pr_auc: 0.9951542038750745
Gini: 0.9898473271650128
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.689125803422157: 


,0,1,Error,Rate
0,1541.0,356.0,0.1877,(356.0/1897.0)
1,220.0,30652.0,0.0071,(220.0/30872.0)
Total,1761.0,31008.0,0.0176,(576.0/32769.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.6891258,0.9906917,195.0
max f2,0.4670463,0.9933452,254.0
max f0point5,0.8712775,0.9929180,131.0
max accuracy,0.6891258,0.9824224,195.0
max precision,0.9829572,1.0,0.0
max recall,0.0575642,1.0,377.0
max specificity,0.9829572,1.0,0.0
max absolute_mcc,0.7499109,0.8381336,178.0
max min_per_class_accuracy,0.9112510,0.9699526,109.0
max mean_per_class_accuracy,0.9269884,0.9732658,98.0


Gains/Lift Table: Avg response rate: 94.21 %, avg score: 93.01 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4198672,0.9743337,285.0
max f2,0.0252146,0.9880550,393.0
max f0point5,0.8639684,0.9698248,149.0
max accuracy,0.5895536,0.9507766,242.0
max precision,0.9825417,0.9910913,2.0
max recall,0.0069894,1.0,399.0
max specificity,0.9838880,0.9994729,0.0
max absolute_mcc,0.8128570,0.4847378,171.0
max min_per_class_accuracy,0.9653976,0.7841086,56.0
max mean_per_class_accuracy,0.9567897,0.7956726,72.0


Gains/Lift Table: Avg response rate: 94.21 %, avg score: 94.21 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100095,0.9825220,1.0517389,1.0517389,0.9908537,0.9830890,0.9908537,0.9830890,0.0105273,0.0105273,5.1738907,5.1738907
,2,0.0200189,0.9819118,1.0323222,1.0420305,0.9725610,0.9821965,0.9817073,0.9826427,0.0103330,0.0208603,3.2322189,4.2030548
,3,0.0300284,0.9815166,1.0420305,1.0420305,0.9817073,0.9816867,0.9817073,0.9823240,0.0104302,0.0312905,4.2030548,4.2030548
,4,0.0400073,0.9811958,1.0419712,1.0420157,0.9816514,0.9813499,0.9816934,0.9820811,0.0103978,0.0416883,4.1971170,4.2015737
,5,0.0500168,0.9809411,1.0517389,1.0439616,0.9908537,0.9810635,0.9835265,0.9818774,0.0105273,0.0522156,5.1738907,4.3961558
,6,0.1000031,0.9800100,1.0504310,1.0471953,0.9896215,0.9804516,0.9865731,0.9811647,0.0525071,0.1047227,5.0431025,4.7195304
,7,0.1500198,0.9792850,1.0530282,1.0491400,0.9920683,0.9796395,0.9884052,0.9806562,0.0526691,0.1573918,5.3028221,4.9140005
,8,0.2000061,0.9786551,1.0471910,1.0486529,0.9865690,0.9789707,0.9879463,0.9802349,0.0523452,0.2097370,4.7190954,4.8652891
,9,0.3000092,0.9773670,1.0501105,1.0491388,0.9893195,0.9780123,0.9884040,0.9794941,0.1050143,0.3147512,5.0110478,4.9138753
,10,0.4000122,0.9759690,1.0504344,1.0494627,0.9896247,0.9766972,0.9887092,0.9787948,0.1050466,0.4197979,5.0434386,4.9462662



<bound method ModelBase.coef_norm of >


## Generating and storing Hyperparameters of each model into JSON

In [45]:
model_set=aml_leaderboard_df['model_id']
type(model_set)

pandas.core.series.Series

In [46]:
jsonDicts = []
for m in model_set.iteritems():
    m,model_name = m
    mod_best = h2o.get_model(model_name)
    jsonDicts.append(mod_best.params)

In [47]:
print(jsonDicts)

[{'model_id': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'StackedEnsemble_AllModels_AutoML_20190418_120619', 'type': 'Key<Model>', 'URL': '/3/Models/StackedEnsemble_AllModels_AutoML_20190418_120619'}}, 'training_frame': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'automl_training_py_6_sid_8fd2', 'type': 'Key<Frame>', 'URL': '/3/Frames/automl_training_py_6_sid_8fd2'}}, 'response_column': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ColSpecifierV3', 'schema_type': 'VecSpecifier'}, 'column_name': 'ACTION', 'is_member_of_frames': None}}, 'validation_frame': {'default': None, 'actual': None}, 'blending_frame': {'default': None, 'actual': None}, 'base_models': {'default': [], 'actual': [{'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GBM_grid

In [48]:
n=run_id+'EmpAccess_'+ str(run_time)+'_hy_parameter.json'
dict_to_json(jsonDicts,n)

In [49]:
jsonDictsActual = []
for m in model_set.iteritems():
    m,model_name = m
    mod_best = h2o.get_model(model_name)
    jsonDictsActual.append(mod_best.actual_params)

In [50]:
print(jsonDictsActual)

[<property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property object at 0x000000AAB8FA54F8>, <property objec

### Generating metadata and storing in JSON

In [51]:
meta_data['end_time'] = time.time()

In [52]:
meta_data

{'project': 'kfOyShRCX_EmpAccess_1200',
 'run_time': 1200,
 'run_id': 'kfOyShRCX',
 'start_time_sec': 1555603544.8401034,
 'min_mem_size': 2,
 'balance': False,
 'balance_threshold': 0.2,
 'max_models': None,
 'scale': False,
 'model_path': None,
 'server_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project',
 'data_path': 'C:/Users/hp/Desktop/Data Science Hyperparameter Project/Employee_access_data.csv',
 'run_path': 'C:\\Users\\hp\\Desktop\\Data Science Hyperparameter Project\\kfOyShRCX_EmpAccess_1200',
 'nthreads': 1,
 'analysis': 0,
 'end_time_sec': 1555603544.8401034,
 'Problem_type': 'Classification',
 'Total Records': 327690,
 'Target': 'ACTION',
 'X': ['RESOURCE',
  'MGR_ID',
  'ROLE_ROLLUP_1',
  'ROLE_ROLLUP_2',
  'ROLE_DEPTNAME',
  'ROLE_TITLE',
  'ROLE_FAMILY_DESC',
  'ROLE_FAMILY',
  'ROLE_CODE'],
 'levels': [['0', '1']],
 'variables': {'ACTION': 'enum',
  'RESOURCE': 'int',
  'MGR_ID': 'int',
  'ROLE_ROLLUP_1': 'int',
  'ROLE_ROLLUP_2': 'int',
  'ROLE_DEPTNA

In [53]:
n=run_id+'EmpAccess_' + str(run_time) + 'meta_data.json'
dict_to_json(meta_data,n)

#### Shutting down H2O

In [55]:
# Clean up
os.chdir(server_path)

In [56]:
h2o.cluster().shutdown()

H2O session _sid_8fd2 closed.


#### Link of data set

https://www.kaggle.com/c/amazon-employee-access-challenge/overview

# Conclusion

# Contribution

percentage ratio - 80: 20

Self - 80%

External - 20%

Urja - 40%

Prakruthi - 40%

# Citation

https://github.com/prabhuSub/Hyperparamter-Samples

https://github.com/nikbearbrown/CSYE_7245/tree/master/H2O



# License

Copyright (c) 2019 Urja Jain, Prakruthi Bagur Suryanarayanaprasad

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.